In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,accuracy_score, \
                            precision_score,recall_score, roc_auc_score,roc_curve,auc
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [6]:
dftrain = pd.read_csv('df_train.csv')
dftest = pd.read_csv('df_test.csv')

In [7]:
dftrain = dftrain.fillna(0)
dftest = dftest.fillna(0)

In [8]:
dftrain_X = dftrain.drop(['id','flag'],axis=1) 
dftrain_Y = dftrain['flag']
dftest_preds = dftest.drop(['id'],axis=1)

In [13]:
from xgboost import XGBClassifier
model = XGBClassifier(colsample_bytree=0.6,max_depth=6,reg_alpha=0.05,subsample=0.7,\
                      objective='binary:logistic',n_job=-1,booster='gbtree',\
                      n_estimators=1000,learning_rate=0.02,gamma=0.7)

folds = KFold(n_splits=5, shuffle=True, random_state=6)
predictions = 0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(dftrain_X, dftrain_Y)):
    print('Fold:', fold_+1)
    tr_x, tr_y = dftrain_X.iloc[trn_idx, :], dftrain_Y[trn_idx]
    vl_x, vl_y = dftrain_X.iloc[val_idx, :], dftrain_Y[val_idx]
    
    model.fit(tr_x, tr_y,
        eval_set=[(vl_x, vl_y)],
        eval_metric='auc',
        early_stopping_rounds=100,
        verbose=50)
    y_prob = model.predict_proba(vl_x)[:,1]
    print(roc_auc_score(vl_y, y_prob))
    predictions += roc_auc_score(vl_y, y_prob)/5
print("最终模型auc:", predictions)

Fold: 1
[0]	validation_0-auc:0.68462
Will train until validation_0-auc hasn't improved in 100 rounds.
[50]	validation_0-auc:0.74485
[100]	validation_0-auc:0.74863
[150]	validation_0-auc:0.75139
[200]	validation_0-auc:0.75297
[250]	validation_0-auc:0.75459
[300]	validation_0-auc:0.75539
[350]	validation_0-auc:0.75638
[400]	validation_0-auc:0.75667
[450]	validation_0-auc:0.75663
[500]	validation_0-auc:0.75706
[550]	validation_0-auc:0.75725
[600]	validation_0-auc:0.75695
Stopping. Best iteration:
[516]	validation_0-auc:0.75740

0.7574025718725343
Fold: 2
[0]	validation_0-auc:0.69072
Will train until validation_0-auc hasn't improved in 100 rounds.
[50]	validation_0-auc:0.74706
[100]	validation_0-auc:0.75084
[150]	validation_0-auc:0.75575
[200]	validation_0-auc:0.75707
[250]	validation_0-auc:0.75755
[300]	validation_0-auc:0.75880
[350]	validation_0-auc:0.75826
[400]	validation_0-auc:0.75893
[450]	validation_0-auc:0.75919
[500]	validation_0-auc:0.75993
[550]	validation_0-auc:0.76093
[600]	va

In [14]:
pred = dftest[['id']]
pred['pred_prob'] = model.predict_proba(dftest_preds)[:,1]

In [ ]:
with open('sumbit.txt',mode ='w',encoding='utf-8') as f:
    for line in pred.values:
        f.write(str(line[0])+'\t'+str(line[1])+'\n')